<a href="https://colab.research.google.com/github/adityamishra5050/Coronavirus-tweets-NLP---Text-Classification-/blob/main/NLU_ass_1_task4(b)(c).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **1-layer LSTM model**

In [ ]:
import pandas as pd
import numpy as np
import re
from sklearn.model_selection import train_test_split


import matplotlib.pyplot as plt
from wordcloud import WordCloud
import seaborn as sns


from tensorflow.keras.layers import Dense, Input, LSTM, Embedding, Dropout
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.layers import GlobalMaxPool1D
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.models import Model


from sklearn.metrics import classification_report


import warnings
import os

In [ ]:
Corona_NLP_train = pd.read_csv('/content/drive/MyDrive/Corona_NLP_train.csv', encoding='latin1')
Corona_NLP_test = pd.read_csv('/content/drive/MyDrive/Corona_NLP_test.csv', encoding='latin1')

In [ ]:
# Combaine train and test set
df = Corona_NLP_train.append(Corona_NLP_test, ignore_index=True)

In [ ]:
# Convert sentiment into Positive = 2 , Neutral = 1 , Negative =  0
def replace_lable(sentiment):
    if sentiment == "Extremely Positive":
        return 2
    elif sentiment == "Extremely Negative":
        return 0
    elif sentiment == "Positive":
        return 2
    elif sentiment == "Negative":
        return 0
    else:
        return 1

In [ ]:
# Apply replace_lable function
df.Sentiment = df.Sentiment.apply(lambda x : replace_lable(x))

In [ ]:
# Clean text

def remove_url(text):
    url = re.compile(r'https?://\S+|www\.\S+')
    return url.sub(r'',text)

def remove_punct(text):
    table = str.maketrans('', '', string.punctuation)
    return text.translate(table)
def remove_html(text):
    html=re.compile(r'<.*?>')
    return html.sub(r'',text)

def remove_emoji(text):
    emoji_pattern = re.compile("["
                           u"\U0001F600-\U0001F64F"  # emoticons
                           u"\U0001F300-\U0001F5FF"  # symbols & pictographs
                           u"\U0001F680-\U0001F6FF"  # transport & map symbols
                           u"\U0001F1E0-\U0001F1FF"  # flags (iOS)
                           u"\U00002702-\U000027B0"
                           u"\U000024C2-\U0001F251"
                           "]+", flags=re.UNICODE)
    return emoji_pattern.sub(r'', text)

def decontraction(text):
    text = re.sub(r"won\'t", " will not", text)
    text = re.sub(r"won\'t've", " will not have", text)
    text = re.sub(r"can\'t", " can not", text)
    text = re.sub(r"don\'t", " do not", text)

    text = re.sub(r"can\'t've", " can not have", text)
    text = re.sub(r"ma\'am", " madam", text)
    text = re.sub(r"let\'s", " let us", text)
    text = re.sub(r"ain\'t", " am not", text)
    text = re.sub(r"shan\'t", " shall not", text)
    text = re.sub(r"sha\n't", " shall not", text)
    text = re.sub(r"o\'clock", " of the clock", text)
    text = re.sub(r"y\'all", " you all", text)

    text = re.sub(r"n\'t", " not", text)
    text = re.sub(r"n\'t've", " not have", text)
    text = re.sub(r"\'re", " are", text)
    text = re.sub(r"\'s", " is", text)
    text = re.sub(r"\'d", " would", text)
    text = re.sub(r"\'d've", " would have", text)
    text = re.sub(r"\'ll", " will", text)
    text = re.sub(r"\'ll've", " will have", text)
    text = re.sub(r"\'t", " not", text)
    text = re.sub(r"\'ve", " have", text)
    text = re.sub(r"\'m", " am", text)
    text = re.sub(r"\'re", " are", text)
    return text

def seperate_alphanumeric(text):
    words = text
    words = re.findall(r"[^\W\d_]+|\d+", words)
    return " ".join(words)

def cont_rep_char(text):
    tchr = text.group(0)

    if len(tchr) > 1:
        return tchr[0:2]

def unique_char(rep, text):
    substitute = re.sub(r'(\w)\1+', rep, text)
    return substitute

def char(text):
    substitute = re.sub(r'[^a-zA-Z]',' ',text)
    return substitute

# Apply functions on OriginalTweet
df['OriginalTweet'] = df['OriginalTweet'].apply(lambda x : remove_url(x))
df['OriginalTweet'] = df['OriginalTweet'].apply(lambda x : remove_emoji(x))
df['OriginalTweet'] = df['OriginalTweet'].apply(lambda x : decontraction(x))
df['OriginalTweet'] = df['OriginalTweet'].apply(lambda x : seperate_alphanumeric(x))
df['OriginalTweet'] = df['OriginalTweet'].apply(lambda x : unique_char(cont_rep_char,x))
df['OriginalTweet'] = df['OriginalTweet'].apply(lambda x : char(x))
df['OriginalTweet'] = df['OriginalTweet'].apply(lambda x : x.lower())


In [ ]:
# Split sentiment groups
Positive = df[df['Sentiment'] == 2].OriginalTweet
Neutral  = df[df['Sentiment'] == 1].OriginalTweet
Negative = df[df['Sentiment'] == 0].OriginalTweet

In [ ]:
corona_X = df['OriginalTweet']
corona_y = df['Sentiment']

# Split data into train , test
corona_X_train , corona_X_test , corona_y_train , corona_y_test = train_test_split(corona_X,corona_y,test_size=0.2,random_state=0)

In [ ]:
print('Shape of corona_X_train : ' , corona_X_train.shape)
print('Shape of corona_y_train : ', corona_y_train.shape)
print('Shape of corona_X_test : ', corona_X_test.shape)
print('Shape of corona_y_test : ',corona_y_test.shape)

Shape of corona_X_train :  (35964,)
Shape of corona_y_train :  (35964,)
Shape of corona_X_test :  (8991,)
Shape of corona_y_test :  (8991,)


In [ ]:
# Convert y to categorical
corona_y_train = to_categorical(corona_y_train, 3)
corona_y_test  = to_categorical(corona_y_test, 3)

In [ ]:
# Tokenize Texts
max_features = 20000
tokenizer = Tokenizer(num_words=max_features)
tokenizer.fit_on_texts(list(corona_X_train))
list_tokenized_train = tokenizer.texts_to_sequences(corona_X_train)
list_tokenized_test = tokenizer.texts_to_sequences(corona_X_test)

In [ ]:
# convert tokenized texts into same padding size
maxlen = 200
embed_size = 128
corona_X_train_final = pad_sequences(list_tokenized_train, maxlen=maxlen)
corona_X_test_final = pad_sequences(list_tokenized_test, maxlen=maxlen)

In [ ]:
# Create Model
inp = Input(shape=(maxlen, ))
x   =  Embedding(max_features, embed_size)(inp)
x   =  LSTM(60, return_sequences=True,name='lstm_layer')(x)
x   =  GlobalMaxPool1D()(x)
x   =  Dense(64, activation="relu")(x)
x   =  Dropout(0.2)(x)
x   =  Dense(3, activation="softmax")(x)

In [ ]:
model_1layer = Model(inputs=inp, outputs=x)
model_1layer.compile(loss='categorical_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])

In [ ]:
history = model_1layer.fit(corona_X_train_final,corona_y_train, epochs=2,validation_data=(corona_X_test_final,corona_y_test))

Epoch 1/2
1124/1124 [==============================] - 281s 247ms/step - loss: 0.6011 - accuracy: 0.7534 - val_loss: 0.4240 - val_accuracy: 0.8546
Epoch 2/2
1124/1124 [==============================] - 282s 251ms/step - loss: 0.3027 - accuracy: 0.9002 - val_loss: 0.3682 - val_accuracy: 0.8745


In [ ]:
model_1layer.evaluate(corona_X_test_final,corona_y_test)

281/281 [==============================] - 14s 49ms/step - loss: 0.3682 - accuracy: 0.8745


[0.368225634098053, 0.8745412230491638]

In [ ]:
import plotly.express as px

In [ ]:
fig = px.line(
    history.history,
    y=['loss', 'val_loss'],
    labels={'index': "epoch", 'value': "loss"},width=800, height=400
)

fig.show()

In [ ]:
fig = px.line(
    history.history, y=['accuracy', 'val_accuracy'],
    labels={'index': 'epoch', 'value': 'accuracy'},width=800, height=400
)

fig.show()

# **2-layer LSTM model**

In [ ]:
# Create Model
inp1 = Input(shape=(maxlen, ))
x1   =  Embedding(max_features, embed_size)(inp1)
x1  =  LSTM(60, return_sequences=True,name='lstm_layer')(x1)
x1   =  LSTM(60, return_sequences=True,name='lstm_layer1')(x1)
x1   =  GlobalMaxPool1D()(x1)
x1   =  Dense(64, activation="relu")(x1)
x1   =  Dropout(0.2)(x1)
x1   =  Dense(3, activation="softmax")(x1)

In [ ]:
model_2layer = Model(inputs=inp1, outputs=x1)
model_2layer.compile(loss='categorical_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])

In [ ]:
history1 = model_2layer.fit(corona_X_train_final,corona_y_train, epochs=2,validation_data=(corona_X_test_final,corona_y_test))

Epoch 1/2
1124/1124 [==============================] - 464s 407ms/step - loss: 0.6158 - accuracy: 0.7464 - val_loss: 0.4222 - val_accuracy: 0.8530
Epoch 2/2
1124/1124 [==============================] - 456s 405ms/step - loss: 0.3195 - accuracy: 0.8963 - val_loss: 0.3775 - val_accuracy: 0.8705


In [ ]:
model_2layer.evaluate(corona_X_test_final,corona_y_test)

281/281 [==============================] - 25s 90ms/step - loss: 0.3775 - accuracy: 0.8705


[0.3775195777416229, 0.8705372214317322]

In [ ]:
fig = px.line(
    history1.history,
    y=['loss', 'val_loss'],
    labels={'index': "epoch", 'value': "loss"},width=800, height=400
)

fig.show()

In [ ]:
fig = px.line(
    history1.history, y=['accuracy', 'val_accuracy'],
    labels={'index': 'epoch', 'value': 'accuracy'},width=800, height=400
)

fig.show()